# Geoprocessing for Typhoon Frequency

In [ ]:
# Search engines are your friends!
# Use them to look up the technical documentation
# of each functionality imported here
from geopandas import read_file, GeoDataFrame, sjoin
from pandas import concat

### Data Paths

In [ ]:
tp_fp = 'Luzon_TCTLines_IBTrACS_1980-2020.shp'
grid_fp = 'Luzon_10KmGrid_MO-GED-2022.shp'

tp_df = read_file(tp_fp)
grid_df = read_file(grid_fp)

### Plot Datasets
- Documentation on plotting datasets can be accessed [here](https://geopandas.org/en/stable/docs/user_guide/mapping.html).

In [ ]:
base = grid_df.plot(color='red',zorder=2)
tp_df.plot(ax=base, zorder=1)

### Dissolve by SID and Buffer 100km

In [ ]:
dissolved = tp_df.dissolve(by='SID')

buffered = dissolved.buffer(distance=100000) # meters

In [ ]:
# rename so that column name is 'buffered'
# when concatenated with the dissolved dataset
buffered.rename('buffered',inplace=True) 

### Spatial Join and Count per Grid

In [ ]:
# preparatory steps before joining and counting
# we need to get the datasets into the right
# structure
concatenated = concat([dissolved,buffered],axis=1)
concatenated.set_geometry('buffered',inplace=True) # So that we use the buffer geometry when doing a spatial join
concatenated.reset_index(inplace=True) # To count the number of SIDs within each grid cell

# spatial join operation
joined = grid_df.overlay(concatenated,how='intersection',keep_geom_type=False)

# count per grid operation
counted = joined.groupby('id').SID.count()

counted.rename('counted',inplace=True)

count_per_grid = grid_df.merge(counted, on='id')

### Reclassification

In [ ]:
# Define a function to evaluate
# the count of typhoons per grid
# and return a score
def reclassify(counts):
    if counts>80:
        return 5
    elif counts>=61 and counts<=80:
        return 4
    elif counts>=41 and counts<=60:
        return 3
    elif counts>=18 and counts<=40:
        return 2
    elif counts>=0 and counts<=17:
        return 1

In [ ]:
# The .apply function does the 'looping'
# on each element of the counted column
# and applies the 'reclassify' function
# per element
reclassified = count_per_grid.counted.apply(func=reclassify)

In [ ]:
# restructure for convenience
reclassified.rename('reclassify',inplace=True)

tp_freq = concat([count_per_grid,reclassified],axis=1)

In [ ]:
# saves your selected dataset to
# a shp file
tp_freq.to_file('tp_freq.shp')